### Please run with Google Colab with Good GPU
<a href="https://colab.research.google.com/github/wakachii/SI-Org-chart/blob/main/pipeline/deeplearning_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Detectron2 has not released pre-built binaries for the latest pytorch (https://github.com/facebookresearch/detectron2/issues/4053)
# so we install from source instead. This takes a few minutes.
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Install pre-built detectron2 that matches pytorch version, if released:
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/{CUDA_VERSION}/{TORCH_VERSION}/index.html

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-aqxtuvva
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-aqxtuvva
  Resolved https://github.com/facebookresearch/detectron2.git to commit b1c43ffbc995426a9a6b5c667730091a384e0fa4
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 56.7 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp31

In [4]:
import cv2 as cv2
import json
import os
from google.colab import drive
drive.mount('/content/drive')

# import some common detectron2 utilities
import detectron2
from tqdm import tqdm
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import Visualizer, ColorMode

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
path = "/content/drive/MyDrive/scan_org_charts/learning"
path_train = path + "/data/train"
path_coco = path + "/Org_chart-1.json"
path_data = "/content/drive/MyDrive/scan_org_charts/cropped"
# set train data
register_coco_instances("org_chart_train", {}, path_coco, path_train)

# setting for using the model
cfg = get_cfg() # initialize
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("org_chart_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.BASE_LR = 0.0004
cfg.SOLVER.MAX_ITER = (500)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (128)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

# train
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True) # for output
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[01/10 14:56:48 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f10217.pkl: 178MB [00:01, 106MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


[01/10 14:56:53 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[01/10 14:57:09 d2.utils.events]:  eta: 0:02:08  iter: 19  total_loss: 5.1  loss_cls: 0.6125  loss_box_reg: 0.6728  loss_mask: 0.6881  loss_rpn_cls: 2.824  loss_rpn_loc: 0.3079    time: 0.2823  last_time: 0.2633  data_time: 0.1156  last_data_time: 0.0037   lr: 1.5585e-05  max_mem: 1558M
[01/10 14:57:18 d2.utils.events]:  eta: 0:02:00  iter: 39  total_loss: 2.353  loss_cls: 0.589  loss_box_reg: 0.8397  loss_mask: 0.6271  loss_rpn_cls: 0.06115  loss_rpn_loc: 0.1836    time: 0.2638  last_time: 0.2785  data_time: 0.0053  last_data_time: 0.0180   lr: 3.1569e-05  max_mem: 1558M
[01/10 14:57:23 d2.utils.events]:  eta: 0:01:57  iter: 59  total_loss: 2.011  loss_cls: 0.527  loss_box_reg: 0.8457  loss_mask: 0.5076  loss_rpn_cls: 0.03906  loss_rpn_loc: 0.1976    time: 0.2715  last_time: 0.2369  data_time: 0.0165  last_data_time: 0.0033   lr: 4.7553e-05  max_mem: 1558M
[01/10 14:57:28 d2.utils.events]:  eta: 0:01:44  iter: 79  total_loss: 1.9  loss_cls: 0.4951  loss_box_reg: 0.8149  loss_mask: 0.3

In [7]:
# the function for making the meta-data dict of the test data
def get_test_dicts(img_dir):
    img_files = [os.path.join(img_dir, f) for f in os.listdir(img_dir) if f.endswith('.jpg') or f.endswith('.png')]
    dataset_dicts = []
    for idx, img_file in enumerate(img_files):
        record = {}
        record["file_name"] = img_file
        record["image_id"] = idx
        record["height"], record["width"] = cv2.imread(img_file).shape[:2]
        dataset_dicts.append(record)
    return dataset_dicts

# change the test data form for dectron2
DatasetCatalog.register("org_chart_data", lambda: get_test_dicts(path_data))
MetadataCatalog.get("org_chart_data").set(thing_classes=["department"])

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # load trained weights
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.8  # score
cfg.DATASETS.TEST = ("org_chart_data", )  # set the test data to the model

# detect departments
predictor = DefaultPredictor(cfg)
metadata = MetadataCatalog.get("org_chart_data")
dataset_dicts = DatasetCatalog.get("org_chart_data")

[01/10 15:04:45 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...


KeyboardInterrupt: 

In [ ]:
output_path = "/content/drive/MyDrive/scan_org_charts/learning/output"
for d in tqdm(dataset_dicts):
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)
    json_output = {
    "file_name": d["file_name"],
    "pred_boxes": outputs["instances"].pred_boxes.tensor.cpu().numpy().tolist(),
    "scores": outputs["instances"].scores.cpu().numpy().tolist(),
    "pred_classes": outputs["instances"].pred_classes.cpu().numpy().tolist()
    }
    # save JSON
    base_name = os.path.basename(d["file_name"])
    json_name = base_name.replace(".png", ".json")
    json_path = os.path.join(output_path, json_name)
    with open(json_path, "w") as f:
        json.dump(json_output, f)